This file contains the code used to transform the Medicaid data to a panel data format suitable for running regressions in statsmodels.

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (15,10)

# Transforming Med D Data to Panel Data

In [2]:
medD_spendingbydrug = pd.read_csv("C:\\Users\\ttrap\\Desktop\\ECON 191\\Medicaid_Spending_by_Drug_2021.csv")

In [3]:
medD_spendingbydrug.shape

(16146, 36)

In [4]:
#Create a new dataframe from medD_spendingbydrug that is in a panel data format
medD_spendingbydrug.head()

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Tot_Spndng_2017,Tot_Dsg_Unts_2017,Tot_Clms_2017,Avg_Spnd_Per_Dsg_Unt_Wghtd_2017,Avg_Spnd_Per_Clm_2017,Outlier_Flag_2017,...,Avg_Spnd_Per_Clm_2020,Outlier_Flag_2020,Tot_Spndng_2021,Tot_Dsg_Unts_2021,Tot_Clms_2021,Avg_Spnd_Per_Dsg_Unt_Wghtd_2021,Avg_Spnd_Per_Clm_2021,Outlier_Flag_2021,Chg_Avg_Spnd_Per_Dsg_Unt_20_21,CAGR_Avg_Spnd_Per_Dsg_Unt_17_21
0,8hr Arthritis Pain,Acetaminophen,1,Overall,NaN,NaN,NaN,NaN,NaN,NaN,...,9.281739,1.0,331.69,4642.000,54,0.071454,6.142407,1,0.469539,0.047199
1,8hr Arthritis Pain,Acetaminophen,1,Topco,NaN,NaN,NaN,NaN,NaN,NaN,...,9.281739,1.0,331.69,4642.000,54,0.071454,6.142407,1,-0.253762,0.047199
2,A & D,Vitamins A And D,1,Overall,NaN,NaN,NaN,NaN,NaN,NaN,...,7.866098,1.0,253.76,9136.000,26,0.027776,9.760000,1,0.364446,0.594980
3,A & D,Vitamins A And D,1,Schering-Plough,NaN,NaN,NaN,NaN,NaN,NaN,...,7.866098,1.0,253.76,9136.000,26,0.027776,9.760000,1,0.356607,0.594980
4,A And D,Vits A And D/White Pet/Lanolin,1,Overall,406.33,3502.5,47.0,0.116011,8.645319,1.0,...,7.288947,1.0,106.45,1032.708,14,0.103079,7.603571,1,0.296316,-0.029117


In [5]:
medD_spendingbydrug.columns

Index(['Brnd_Name', 'Gnrc_Name', 'Tot_Mftr', 'Mftr_Name', 'Tot_Spndng_2017',
       'Tot_Dsg_Unts_2017', 'Tot_Clms_2017', 'Avg_Spnd_Per_Dsg_Unt_Wghtd_2017',
       'Avg_Spnd_Per_Clm_2017', 'Outlier_Flag_2017', 'Tot_Spndng_2018',
       'Tot_Dsg_Unts_2018', 'Tot_Clms_2018', 'Avg_Spnd_Per_Dsg_Unt_Wghtd_2018',
       'Avg_Spnd_Per_Clm_2018', 'Outlier_Flag_2018', 'Tot_Spndng_2019',
       'Tot_Dsg_Unts_2019', 'Tot_Clms_2019', 'Avg_Spnd_Per_Dsg_Unt_Wghtd_2019',
       'Avg_Spnd_Per_Clm_2019', 'Outlier_Flag_2019', 'Tot_Spndng_2020',
       'Tot_Dsg_Unts_2020', 'Tot_Clms_2020', 'Avg_Spnd_Per_Dsg_Unt_Wghtd_2020',
       'Avg_Spnd_Per_Clm_2020', 'Outlier_Flag_2020', 'Tot_Spndng_2021',
       'Tot_Dsg_Unts_2021', 'Tot_Clms_2021', 'Avg_Spnd_Per_Dsg_Unt_Wghtd_2021',
       'Avg_Spnd_Per_Clm_2021', 'Outlier_Flag_2021',
       'Chg_Avg_Spnd_Per_Dsg_Unt_20_21', 'CAGR_Avg_Spnd_Per_Dsg_Unt_17_21'],
      dtype='object')

## Task

1. We want to create a new column called 'Year' which has the year that the particular observation was made. 
2. Have only 13 columns total: The 4 identifying columns, Year, Tot_Spndng, Tot_Dsg_Unts, Tot_Clms, Tot_Benes, Avg_Spnd_Per_Dsg_Unt_Wghtd, Avg_Spnd_Per_Clm, Avg_Spnd_Per_Bene, Outlier_Flag. Somehow, we want to merge all of the data into 8 columns, and have the year column to identify the year of each row. 

### Game Plan: Make 8 dataframes with year column and respective data column by exploding a dictionary, then merge all of the dataframes on year.

In [6]:
medD = medD_spendingbydrug

In [7]:
#Create a column of dictionaries, where each key, value is the year, total spending for that year. 
#Test Code: Proof of Concept
years = ['2017', '2018', '2019', '2020', '2021']
total_spend = pd.DataFrame(
    pd.DataFrame(
    pd.Series(
        pd.DataFrame(
            medD[medD.columns[4::6][:5]]).values.tolist())).apply(lambda x: dict(zip(years, x[0])), axis = 1))
total_spend

,0
0,"{'2017': nan, '2018': nan, '2019': 733.42, '20..."
1,"{'2017': nan, '2018': nan, '2019': 733.42, '20..."
2,"{'2017': nan, '2018': 252.22, '2019': 770.31, ..."
3,"{'2017': nan, '2018': 252.22, '2019': 770.31, ..."
4,"{'2017': 406.33, '2018': 213.7, '2019': 79.36,..."
...,...
16141,"{'2017': 881336.15, '2018': 281239.18, '2019':..."
16142,"{'2017': 516194.24, '2018': 236030.6, '2019': ..."
16143,"{'2017': 400349.22, '2018': 170408.66, '2019':..."
16144,"{'2017': 63972.27, '2018': 40694.65, '2019': 2..."


In [8]:
medD[medD.columns[4::6][:5]]

,Tot_Spndng_2017,Tot_Spndng_2018,Tot_Spndng_2019,Tot_Spndng_2020,Tot_Spndng_2021
0,NaN,NaN,733.42,426.96,331.69
1,NaN,NaN,733.42,426.96,331.69
2,NaN,252.22,770.31,322.51,253.76
3,NaN,252.22,770.31,322.51,253.76
4,406.33,213.70,79.36,138.49,106.45
...,...,...,...,...,...
16141,881336.15,281239.18,180994.29,69494.24,22141.27
16142,516194.24,236030.60,149195.31,115718.45,108270.21
16143,400349.22,170408.66,99561.40,84941.16,67237.88
16144,63972.27,40694.65,24742.51,17824.48,17779.03


In [9]:
#Function which creates the dataframe of dictionaries
#Takes in dataframe, list of keys. Assumes that the df is a dataframe of lists
#Also takes in two index values in order to select the correct columns from df
#i = index to start at, k = number to increment by after i
def df_dict(df, keys, i, k = 8):
    df = df.fillna(0)
    dict_df = pd.DataFrame(
    pd.DataFrame(
    pd.Series(
        pd.DataFrame(
            df[df.columns[i::k][:5]]).values.tolist())).apply(lambda x: dict(zip(keys, x[0])), axis = 1))
    return dict_df

In [10]:
#Testing the merge with the primary keys of the data
m = pd.DataFrame([*total_spend[0]], total_spend.index).stack()\
      .rename_axis([None,'year']).reset_index(1, name='total_spending')

out = medD[list(medD.columns)[0:4]].join(m)
out.head(10)

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,year,total_spending
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2019,733.42
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2020,426.96
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2021,331.69
1,8hr Arthritis Pain,Acetaminophen,1,Topco,2019,733.42
1,8hr Arthritis Pain,Acetaminophen,1,Topco,2020,426.96
1,8hr Arthritis Pain,Acetaminophen,1,Topco,2021,331.69
2,A & D,Vitamins A And D,1,Overall,2018,252.22
2,A & D,Vitamins A And D,1,Overall,2019,770.31
2,A & D,Vitamins A And D,1,Overall,2020,322.51
2,A & D,Vitamins A And D,1,Overall,2021,253.76


In [11]:
#Function which explodes column of dictionaries into df
#value_name = name of the type of data in the values of the dictionary, string
def explode_dictionaries(dict_df, value_name):
    return pd.DataFrame([*dict_df[0]], dict_df.index).stack()\
      .rename_axis([None,'Year']).reset_index(1, name = value_name)

In [12]:
explode_dictionaries(total_spend, 'total_spending')

,Year,total_spending
0,2019,733.42
0,2020,426.96
0,2021,331.69
1,2019,733.42
1,2020,426.96
...,...,...
16145,2017,51872.75
16145,2018,24927.29
16145,2019,24891.40
16145,2020,12952.81


In [13]:
#Time-invariant data (Primary keys)
primary_keys = medD[list(medD.columns)[0:4]]
primary_keys.head()

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name
0,8hr Arthritis Pain,Acetaminophen,1,Overall
1,8hr Arthritis Pain,Acetaminophen,1,Topco
2,A & D,Vitamins A And D,1,Overall
3,A & D,Vitamins A And D,1,Schering-Plough
4,A And D,Vits A And D/White Pet/Lanolin,1,Overall


# Creating the medD Panel Data

In [14]:
years = ['2017', '2018', '2019', '2020', '2021']
values = ['Tot_Spndng', 'Tot_Dsg_Unts', 'Tot_Clms', 'Avg_Spnd_Per_Dsg_Unt_Wghtd', 
          'Avg_Spnd_Per_Clm', 'Outlier_Flag']

In [15]:
total_spending = explode_dictionaries(df_dict(medD, years, i = 4, k = 6), values[0])

total_dosage = explode_dictionaries(df_dict(medD, years, i = 5, k = 6), values[1])

total_claims = explode_dictionaries(df_dict(medD, years, i = 6, k = 6), values[2])

avg_spend_dsg = explode_dictionaries(df_dict(medD, years, i = 7, k = 6), values[3])

avg_spend_clm = explode_dictionaries(df_dict(medD, years, i = 8, k = 6), values[4])

outlier_flag = explode_dictionaries(df_dict(medD, years, i = 9, k = 6), values[5])

In [16]:
primary_keys

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name
0,8hr Arthritis Pain,Acetaminophen,1,Overall
1,8hr Arthritis Pain,Acetaminophen,1,Topco
2,A & D,Vitamins A And D,1,Overall
3,A & D,Vitamins A And D,1,Schering-Plough
4,A And D,Vits A And D/White Pet/Lanolin,1,Overall
...,...,...,...,...
16141,Zyvox,Linezolid,1,Pharmaci/Pfizer
16142,Zyvox,Linezolid In Dextrose 5%,3,Overall
16143,Zyvox,Linezolid In Dextrose 5%,1,Pharmaci/Pfizer
16144,Zyvox,Linezolid In Dextrose 5%,1,Phar-Prep/Pfize


In [17]:
years = ['2017', '2018', '2019', '2020', '2021']
values = ['Tot_Spndng', 'Tot_Dsg_Unts', 'Tot_Clms', 'Avg_Spnd_Per_Dsg_Unt_Wghtd', 
          'Avg_Spnd_Per_Clm', 'Outlier_Flag']

In [18]:
medD_panel = primary_keys.join(total_spending)

In [19]:
medD_panel[values[1]] = total_dosage.iloc[:, 1]
medD_panel[values[2]] = total_claims.iloc[:, 1]
medD_panel[values[3]] = avg_spend_dsg.iloc[:, 1]
medD_panel[values[4]] = avg_spend_clm.iloc[:, 1]
medD_panel[values[5]] = outlier_flag.iloc[:, 1]

In [20]:
medD_panel.head()

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Year,Tot_Spndng,Tot_Dsg_Unts,Tot_Clms,Avg_Spnd_Per_Dsg_Unt_Wghtd,Avg_Spnd_Per_Clm,Outlier_Flag
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2017,0.00,0.0,0.0,0.000000,0.000000,0.0
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2018,0.00,0.0,0.0,0.000000,0.000000,0.0
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2019,733.42,11256.0,140.0,0.065158,5.238714,1.0
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2020,426.96,4459.0,46.0,0.095752,9.281739,1.0
0,8hr Arthritis Pain,Acetaminophen,1,Overall,2021,331.69,4642.0,54.0,0.071454,6.142407,1.0


In [21]:
#Write to CSV 
#medD_panel.to_csv("C:\\Users\\ttrap\\Desktop\\ECON 191\\med_panel.csv", index = False)